In [28]:
import re
import os
import groq as Groq
from dotenv import load_dotenv

In [29]:
#para rodar tudo, é necessário ter uma chave API do Groq, que pode ser obtida em https://groq.io/developers
#a minha está em um arquivo dotenv para que não seja exposta, mas você pode colocar a sua diretamente no código
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [30]:
client = Groq.Client(
    api_key=groq_api_key,
)

#note que o groq pega uma lista de mensagens, não uma única string
chat_completion = client.chat.completions.create(
    #primeiro parametro é a lista de mensagens
    messages=[

        {

            "role": "user",

            "content": "Explain the importance of fast language models",

        }

    ],
    #segundo parametro é o modelo de linguagem que será usado (é possível setar outro modelo que a API hospeda)
    model="llama-3.3-70b-versatile",
    
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in natural language processing (NLP) as they enable efficient and effective processing of language-related tasks. The importance of fast language models can be understood from the following perspectives:

1. **Real-time Applications**: Fast language models are essential for real-time applications, such as chatbots, virtual assistants, and live language translation. These models need to process and respond to user input quickly, often in a matter of milliseconds.
2. **Scalability**: With the increasing amount of text data being generated, fast language models can handle large volumes of data without significant performance degradation. This scalability is vital for applications that involve processing vast amounts of text data, such as text classification, sentiment analysis, and information retrieval.
3. **Low Latency**: Fast language models minimize latency, which is critical for applications that require immediate feedback, such as language translatio

In [31]:
class Agente:
    def __init__(self, client, sistema):
        self.client = client
        self.sistema = sistema
        self.mensagem = []
        self.mensagem.append({
            "role": "system",
            "content": self.sistema
        })

    def __chamada__(self, mensagem=""):
        if mensagem:
            self.mensagem.append({
                "role": "user",
                "content": mensagem
            })
        resultado = self.executar()
        self.mensagem.append({
            "role": "assistant",
            "content": resultado
        })
        return resultado

    def executar(self):
        return self.client.chat.completions.create(
            messages=self.mensagem,
            model='llama-3.3-70b-versatile'
        )

    def __call__(self, mensagem=""):
        return self.__chamada__(mensagem)


In [32]:
system_prompt = """
Você vai operar em um loop de Pensamento, PAUSE, Ação, PAUSE, Observação, PAUSE.
Ao final do loop, você emitirá uma Resposta.
Use "Pensamento" para descrever seus pensamentos sobre a pergunta que foi feita, retorne PAUSE.
Use "Ação" para executar uma das ações disponíveis e, em seguida, retorne PAUSE.
"Observação" será o resultado da execução dessa ação.

Suas ações disponíveis são:

get_scale:
e.g. get_scale: "Celsius e Fahrenheit" 
Obtém as escalas de temperatura.

transform:
e.g. transform: "Celsius para Fahrenheit"
Transforma uma unidade de temperatura para outra.

calculate:
e.g. calculate: 100 * 1.8 + 32 
Calcula o numero que foi pedido com base na formula de transformação de cada unidade de temperatura. Nesse caso foi transformar 100 graus Celsius em Fahrenheit.

Exemplo de sessão:
Pergunta: Qual é a temperatura de 100 graus Celsius em Fahrenheit, multiplicado por 2?
Pensamento: Preciso transformar 100 graus Celsius em Fahrenheit.
Ação: transform: "Celsius to Fahrenheit"
PAUSE

Você será chamado novamente

(Observação recebida: 100 graus Celsius é igual a 212 graus Fahrenheit.)

Pensamento: Agora preciso multiplicar 212 por 2.
Ação: calculate: 212 * 2
PAUSE

Se você tiver a resposta, retorne-a como Resposta.

Resposta: 424

Agora é sua vez!
""".strip()


In [33]:
def get_scale(unidade_str):
    unidade = unidade_str.split(" e ")
    if len(unidade) != 2:
        return "Por favor, insira duas unidades de temperatura separadas por 'e'."
    return f"Você escolheu as unidades de temperatura {unidade[0]} e {unidade[1]}."


def transform(transformacao_str, value):
    s = transformacao_str.lower().strip()
    if s == "celsius para fahrenheit":
        return value * 1.8 + 32
    elif s == "fahrenheit para celsius":
        return (value - 32) / 1.8
    elif s == "celsius para kelvin":
        return value + 273.15
    elif s == "kelvin para celsius":
        return value - 273.15
    elif s == "fahrenheit para kelvin":
        return (value - 32) / 1.8 + 273.15
    elif s == "kelvin para fahrenheit":
        return (value - 273.15) * 1.8 + 32
    else:
        raise ValueError("Unidade de temperatura inválida.")
    

def calculate(formula):
    return eval(formula)

In [34]:
joao = Agente(client, system_prompt)

In [36]:
resultado = joao("Qual a transformação de 100 graus Celsius para Fahrenheit, multiplicado por 5?")

print(resultado)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.2' : for 'role:assistant' the following must be satisfied[('messages.2.content' : value must be a string)]", 'type': 'invalid_request_error'}}